In [1]:
import inspect

def cleanup(name=None):
    """Delete all functions in the global scope whose names start with 'test_'."""
    if name:
        if name in globals():
            print(f"Deleting function: {name}")
            del globals()[name]
        return
    
    to_delete = [name for name, obj in globals().items()
                if name.startswith("test_") and inspect.isfunction(obj)]


    for name in to_delete:
        print(f"Deleting function: {name}")
        del globals()[name]
        
cleanup()

# Unit Tests mit pytest

## Lernziele

In dieser Session wirst du lernen, wie du professionelle Unit Tests mit pytest schreibst. Wir schauen uns an, wie du Tests strukturierst, Fixtures verwendest und parametrisierte Tests erstellst. Am Ende der Session wirst du in der Lage sein, robusten, wartbaren und aussagekräftigen Testcode zu schreiben.

---

## 1. Was sind Unit Tests?

Stell dir vor, du baust ein Haus. Würdest du erst das komplette Haus bauen und dann prüfen, ob die Fundamente stabil sind? Natürlich nicht! Du würdest jeden Teil während des Baus testen. Genau das machen **Unit Tests** für deinen Code.

Ein Unit Test ist ein automatisierter Test, der eine kleine, isolierte Einheit deines Codes überprüft – meist eine einzelne Funktion oder Methode. Das Ziel ist sicherzustellen, dass jeder Teil deines Codes genau das tut, was er soll.

### Warum Unit Tests schreiben?

- **Frühe Fehlererkennung**: Bugs werden sofort entdeckt, nicht erst beim Kunden
- **Refactoring-Sicherheit**: Du kannst Code umbauen, ohne Angst zu haben, etwas zu brechen
- **Dokumentation**: Tests zeigen, wie dein Code verwendet werden soll
- **Besseres Design**: Testbarer Code ist oft auch besser strukturierter Code

### Warum pytest?

Python hat ein eingebautes Test-Framework (`unittest`), aber pytest ist zum Standard geworden, weil es:

- **Einfacher** ist: Normale `assert`-Statements statt spezieller Methoden
- **Mächtiger** ist: Fixtures, Parametrisierung, Plugins
- **Besseres Feedback** gibt: Detaillierte Fehlermeldungen
- **Weniger Boilerplate** braucht: Keine Test-Klassen erforderlich

---

## 2. Deine ersten Tests mit pytest


### Ein einfacher Test

Ein pytest-Test ist einfach eine Funktion, deren Name mit `test_` beginnt und die `assert`-Statements enthält:

In [2]:
from vector import Vector

def test_vector_creation():
    """Testet die Erstellung eines Vektors"""
    v = Vector([1, 2, 3])
    assert len(v) == 3
    assert v[0] == 1.0
    assert v[1] == 2.0
    assert v[2] == 3.0

def test_vector_addition():
    """Testet die Addition zweier Vektoren"""
    v1 = Vector([1, 2, 3])
    v2 = Vector([4, 5, 6])
    result = v1 + v2
    assert result == Vector([5, 7, 9])

In [3]:
test_vector_creation()
test_vector_addition()

### Tests in Jupyter ausführen

Für Jupyter Notebooks verwenden wir `ipytest`, das pytest in Notebooks integriert:

In [4]:
import ipytest
ipytest.autoconfig() # Konfiguriert ipytest mit sinnvollen Standardeinstellungen


ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 2 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_creation PASSED                           [ 50%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition PASSED                           [100%]

======================================== 2 passed in 0.13s ========================================


<ExitCode.OK: 0>

Die `-v` Flag steht für "verbose" und gibt uns detaillierte Ausgaben.

### Was passiert bei einem Fehlschlag?

In [5]:
def test_vector_addition_wrong():
    v1 = Vector([1, 2])
    v2 = Vector([3, 4])
    # Dieser Test schlägt fehl - absichtlicher Fehler!
    assert v1 + v2 == Vector([4, 5])  # Sollte [4, 6] sein

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 3 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_creation PASSED                           [ 33%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition PASSED                           [ 66%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition_wrong FAILED                     [100%]

============================================ FAILURES =============================================
___________________________________ test_vector_addition_wrong ____________________________________

    def test_vector_addition_wrong():
        v1 = Vector([1, 2])
        v2 = Vector([3, 

<ExitCode.TESTS_FAILED: 1>

In [6]:
cleanup("test_vector_addition_wrong") # Entfernt den fehlerhaften Test
ipytest.run('-vv')

Deleting function: test_vector_addition_wrong
======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 2 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_creation PASSED                           [ 50%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition PASSED                           [100%]

======================================== 2 passed in 0.11s ========================================


<ExitCode.OK: 0>

pytest zeigt uns genau, was schiefgelaufen ist:
- Welcher Test fehlgeschlagen ist
- In welcher Zeile der Fehler auftrat
- Was der erwartete und der tatsächliche Wert war
- Dank `__repr__` sehen wir die Vector-Komponenten klar

---

## 3. Gute Tests schreiben: Das AAA-Muster

Professionelle Tests folgen meist dem **AAA-Muster** (Arrange-Act-Assert):

1. **Arrange**: Bereite die Testdaten und den Kontext vor
2. **Act**: Führe die zu testende Aktion aus
3. **Assert**: Überprüfe, ob das Ergebnis stimmt

In [7]:
cleanup()

def test_vector_normalization():
    # Arrange: Testdaten vorbereiten
    v = Vector([3, 4])
    expected_magnitude = 1.0
    
    # Act: Funktion aufrufen
    normalized = v.normalize()
    
    # Assert: Ergebnis prüfen
    assert abs(normalized) == expected_magnitude
    assert normalized == Vector([0.6, 0.8])

def test_vector_dot_product():
    # Arrange
    v1 = Vector([1, 2, 3])
    v2 = Vector([4, 5, 6])
    
    # Act
    result = v1 @ v2
    
    # Assert
    assert result == 32.0

ipytest.run('-vv')

Deleting function: test_vector_creation
Deleting function: test_vector_addition
======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 2 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_normalization PASSED                      [ 50%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_dot_product PASSED                        [100%]

======================================== 2 passed in 0.24s ========================================


<ExitCode.OK: 0>

Das AAA-Muster macht Tests lesbar und wartbar. Jeder kann auf den ersten Blick verstehen, was getestet wird.

---

## 4. Exceptions testen mit pytest.raises

Manchmal soll dein Code eine Exception werfen. Auch das kannst du testen:

In [8]:
import pytest

def test_vector_addition_dimension_mismatch():
    """Testet, dass Addition verschiedener Dimensionen einen Fehler wirft"""
    v1 = Vector([1, 2, 3])
    v2 = Vector([1, 2])
    
    with pytest.raises(ValueError):
        v1 + v2

def test_vector_zero_division():
    """Testet Division durch Null"""
    v = Vector([1, 2, 3])
    
    with pytest.raises(ValueError):
        v / 0

def test_vector_normalize_zero():
    """Testet, dass Normalisierung eines Nullvektors fehlschlägt"""
    zero_vector = Vector([0, 0, 0])
    
    with pytest.raises(ValueError):
        zero_vector.normalize()

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 5 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_normalization PASSED                      [ 20%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_dot_product PASSED                        [ 40%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition_dimension_mismatch PASSED        [ 60%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_zero_division PASSED                      [ 80%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_normalize_zero PASSED                     [100%]

======================================== 5 passed in 0.25s ================================

<ExitCode.OK: 0>



## 5. Parametrisierte Tests: Ein Test, viele Fälle

Stell dir vor, du möchtest dieselbe Funktion mit vielen verschiedenen Eingaben testen. Du könntest für jeden Fall einen eigenen Test schreiben:

In [9]:
def test_vector_addition_case_1():
    assert Vector([1, 2]) + Vector([3, 4]) == Vector([4, 6])

def test_vector_addition_case_2():
    assert Vector([0, 0]) + Vector([1, 1]) == Vector([1, 1])

def test_vector_addition_case_3():
    assert Vector([-1, -2]) + Vector([1, 2]) == Vector([0, 0])

Das wird schnell repetitiv. Besser ist **Parametrisierung**:

In [10]:
@pytest.mark.parametrize(
    "v1_components, v2_components, expected_components",
    [
        ([1, 2], [3, 4], [4, 6]),
        ([0, 0], [1, 1], [1, 1]),
        ([-1, -2], [1, 2], [0, 0]),
        ([1, 2, 3], [4, 5, 6], [5, 7, 9]),
        ([0.5, 1.5], [0.5, 0.5], [1.0, 2.0]),
    ]
)
def test_vector_addition_parametrized(v1_components, v2_components, expected_components):
    v1 = Vector(v1_components)
    v2 = Vector(v2_components)
    expected = Vector(expected_components)
    assert v1 + v2 == expected

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 13 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_normalization PASSED                      [  7%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_dot_product PASSED                        [ 15%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition_dimension_mismatch PASSED        [ 23%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_zero_division PASSED                      [ 30%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_normalize_zero PASSED                     [ 38%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_addition_case_1 PASSED                  

<ExitCode.OK: 0>

In [11]:
cleanup()

Deleting function: test_vector_normalization
Deleting function: test_vector_dot_product
Deleting function: test_vector_addition_dimension_mismatch
Deleting function: test_vector_zero_division
Deleting function: test_vector_normalize_zero
Deleting function: test_vector_addition_case_1
Deleting function: test_vector_addition_case_2
Deleting function: test_vector_addition_case_3
Deleting function: test_vector_addition_parametrized


Ein Test, fünf Testfälle! pytest führt den Test für jede Parameterkombination aus und zeigt jeden Fall einzeln im Ergebnis an.

### Komplexere Parametrisierung

Du kannst auch komplexe Szenarien parametrisieren und mit aussagekräftigen IDs versehen:

In [12]:
import math

@pytest.mark.parametrize(
    "components, expected_magnitude",
    [
        ([3, 4], 5.0),
        ([1, 0], 1.0),
        ([0, 1], 1.0),
        ([1, 1, 1], math.sqrt(3)),
        ([2, 2, 1], 3.0),
    ],
    ids=["3-4-5_triangle", "unit_x", "unit_y", "3d_unit_diagonal", "pythagorean"]
)
def test_vector_magnitude(components, expected_magnitude):
    v = Vector(components)
    assert abs(v) == pytest.approx(expected_magnitude)

@pytest.mark.parametrize(
    "v1, v2, expected_dot",
    [
        (Vector([1, 0]), Vector([0, 1]), 0.0),      # Orthogonal
        (Vector([1, 0]), Vector([1, 0]), 1.0),      # Parallel
        (Vector([1, 2, 3]), Vector([4, 5, 6]), 32.0),  # Standard
        (Vector([2, 2]), Vector([3, 3]), 12.0),     # Parallel scaled
    ],
    ids=["orthogonal", "parallel_unit", "3d_standard", "parallel_scaled"]
)
def test_vector_dot_product(v1, v2, expected_dot):
    assert v1 @ v2 == pytest.approx(expected_dot)

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 9 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_magnitude[3-4-5_triangle] PASSED          [ 11%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_magnitude[unit_x] PASSED                  [ 22%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_magnitude[unit_y] PASSED                  [ 33%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_magnitude[3d_unit_diagonal] PASSED        [ 44%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_magnitude[pythagorean] PASSED             [ 55%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_vector_dot_product[orthogonal] PASSED           

<ExitCode.OK: 0>

Der `ids`-Parameter gibt jedem Testfall einen aussagekräftigen Namen, der in der Testausgabe erscheint.

---

## 6. Fixtures: Setup und Teardown elegant gelöst

Oft brauchen mehrere Tests dieselben Vorbereitungen: Test-Vektoren, komplexe Objekte oder Setup-Daten. **Fixtures** sind die pytest-Lösung dafür.

### Eine einfache Fixture

In [13]:
cleanup()

@pytest.fixture
def unit_vectors_2d():
    """Erstellt Standard-Einheitsvektoren in 2D"""
    return {
        'x': Vector([1, 0]),
        'y': Vector([0, 1]),
        'diagonal': Vector([1, 1])
    }

def test_unit_vector_magnitude(unit_vectors_2d):
    assert abs(unit_vectors_2d['x']) == 1.0
    assert abs(unit_vectors_2d['y']) == 1.0

def test_unit_vector_orthogonality(unit_vectors_2d):
    # x und y sollten orthogonal sein (dot product = 0)
    assert unit_vectors_2d['x'] @ unit_vectors_2d['y'] == 0.0

ipytest.run('-vv')

Deleting function: test_vector_magnitude
Deleting function: test_vector_dot_product
======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 2 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_magnitude PASSED                     [ 50%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_orthogonality PASSED                 [100%]

======================================== 2 passed in 0.08s ========================================


<ExitCode.OK: 0>

Die Fixture-Funktion wird automatisch vor jedem Test ausgeführt, der sie als Parameter anfordert. Jeder Test bekommt eine frische Kopie.

### Fixtures mit Setup und Teardown

Fixtures können auch Aufräumarbeiten durchführen. Hier ein Beispiel mit temporären Vektordaten:

In [14]:
@pytest.fixture
def vector_test_environment():
    """Erstellt eine Testumgebung mit verschiedenen Vektoren"""
    # Setup: Testvektoren erstellen
    print("\n[Setup] Erstelle Testvektoren")
    vectors = {
        'zero': Vector([0, 0, 0]),
        'unit_x': Vector([1, 0, 0]),
        'unit_y': Vector([0, 1, 0]),
        'unit_z': Vector([0, 0, 1]),
        'standard': Vector([1, 2, 3])
    }
    
    # Die Fixture gibt das Dictionary zurück
    yield vectors
    
    # Teardown: Aufräumen (wird nach dem Test ausgeführt)
    print("\n[Teardown] Räume Testumgebung auf")
    vectors.clear()

def test_zero_vector_properties(vector_test_environment):
    zero = vector_test_environment['zero']
    assert abs(zero) == 0.0
    assert not bool(zero)

def test_unit_vectors_orthogonal(vector_test_environment):
    x = vector_test_environment['unit_x']
    y = vector_test_environment['unit_y']
    z = vector_test_environment['unit_z']
    
    assert x @ y == 0.0
    assert y @ z == 0.0
    assert x @ z == 0.0

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 4 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_magnitude PASSED                     [ 25%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_orthogonality PASSED                 [ 50%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_zero_vector_properties PASSED                    [ 75%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vectors_orthogonal PASSED                   [100%]

======================================== 4 passed in 0.08s ========================================


<ExitCode.OK: 0>

Das `yield`-Statement teilt die Fixture in Setup (davor) und Teardown (danach). Das Teardown wird **immer** ausgeführt, auch wenn der Test fehlschlägt!

### Fixture Scopes

Standardmäßig wird eine Fixture für jeden Test neu erstellt. Du kannst aber den Scope ändern:

In [15]:
@pytest.fixture(scope="module")
def standard_basis_3d():
    """Wird einmal pro Modul erstellt - teuer zu berechnen"""
    print("\n[Setup] Erstelle Standard-Basis (einmal pro Modul)")
    return [
        Vector([1, 0, 0]),
        Vector([0, 1, 0]),
        Vector([0, 0, 1])
    ]

@pytest.fixture(scope="function")  # Standard
def test_vector():
    """Wird für jeden Test neu erstellt"""
    print("\n[Setup] Erstelle Test-Vektor")
    return Vector([1, 2, 3])

def test_basis_length(standard_basis_3d):
    assert len(standard_basis_3d) == 3

def test_basis_orthogonality(standard_basis_3d):
    e1, e2, e3 = standard_basis_3d
    assert e1 @ e2 == 0.0
    assert e2 @ e3 == 0.0
    assert e1 @ e3 == 0.0

def test_vector_modification_isolation(test_vector):
    # Änderungen beeinflussen nicht andere Tests
    original_length = len(test_vector)
    assert original_length == 3

ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0 -- c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collecting ... collected 7 items

t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_magnitude PASSED                     [ 14%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vector_orthogonality PASSED                 [ 28%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_zero_vector_properties PASSED                    [ 42%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_unit_vectors_orthogonal PASSED                   [ 57%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_basis_length PASSED                              [ 71%]
t_c28f7427613c42cf86ac12d88a38b3a3.py::test_basis_orthogonality PASSED                      

<ExitCode.OK: 0>

Scopes:
- `function`: Neu für jeden Test (Standard)
- `class`: Neu für jede Testklasse
- `module`: Neu für jedes Modul/Datei
- `session`: Einmal für die gesamte Testsuite

---

## 7. Testklassen: Tests organisieren

Für komplexere Szenarien kannst du Tests in Klassen gruppieren. Das ist perfekt für die Vector-Klasse:

In [16]:
cleanup()

import pytest
from vector import Vector
import math

class TestVectorArithmetic:
    """Tests für arithmetische Operationen mit Vektoren"""
    
    @pytest.fixture
    def vectors_2d(self):
        """Fixture für 2D-Testvektoren"""
        return {
            'v1': Vector([3, 4]),
            'v2': Vector([1, 2]),
            'zero': Vector([0, 0])
        }
    
    def test_addition(self, vectors_2d):
        result = vectors_2d['v1'] + vectors_2d['v2']
        assert result == Vector([4, 6])
    
    def test_subtraction(self, vectors_2d):
        result = vectors_2d['v1'] - vectors_2d['v2']
        assert result == Vector([2, 2])
    
    def test_scalar_multiplication(self, vectors_2d):
        result = vectors_2d['v1'] * 2
        assert result == Vector([6, 8])
    
    def test_addition_with_zero(self, vectors_2d):
        result = vectors_2d['v1'] + vectors_2d['zero']
        assert result == vectors_2d['v1']
    
    @pytest.mark.parametrize(
        "scalar",
        [0, 1, -1, 2.5, -3.7]
    )
    def test_scalar_multiplication_parametrized(self, vectors_2d, scalar):
        v = vectors_2d['v1']
        result = v * scalar
        expected = Vector([c * scalar for c in v])
        assert result == expected


class TestVectorComparison:
    """Tests für Vergleichsoperationen"""
    
    def test_equality_same_components(self):
        v1 = Vector([1, 2, 3])
        v2 = Vector([1, 2, 3])
        assert v1 == v2
    
    def test_equality_different_components(self):
        v1 = Vector([1, 2, 3])
        v2 = Vector([1, 2, 4])
        assert v1 != v2
    
    def test_less_than_by_magnitude(self):
        short = Vector([1, 1])  # magnitude: ~1.41
        long = Vector([3, 4])   # magnitude: 5
        assert short < long
    
    def test_hashable(self):
        v1 = Vector([1, 2, 3])
        v2 = Vector([1, 2, 3])
        vector_set = {v1, v2}  # Should only have one element
        assert len(vector_set) == 1


class TestVectorAdvanced:
    """Tests für fortgeschrittene Vektoroperationen"""
    
    def test_normalization_standard(self):
        v = Vector([3, 4])
        normalized = v.normalize()
        assert abs(normalized) == pytest.approx(1.0)
        assert normalized == Vector([0.6, 0.8])
    
    def test_distance_calculation(self):
        v1 = Vector([0, 0])
        v2 = Vector([3, 4])
        distance = v1.distance_to(v2)
        assert distance == pytest.approx(5.0)
    
    def test_angle_orthogonal_vectors(self):
        v1 = Vector([1, 0])
        v2 = Vector([0, 1])
        angle = v1.angle_to(v2)
        assert angle == pytest.approx(math.pi / 2)  # 90 degrees
    
    def test_angle_parallel_vectors(self):
        v1 = Vector([1, 0])
        v2 = Vector([2, 0])
        angle = v1.angle_to(v2)
        assert angle == pytest.approx(0.0)

ipytest.run('-v')

Deleting function: test_unit_vector_magnitude
Deleting function: test_unit_vector_orthogonality
Deleting function: test_zero_vector_properties
Deleting function: test_unit_vectors_orthogonal
Deleting function: test_basis_length
Deleting function: test_basis_orthogonality
Deleting function: test_vector_modification_isolation
======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collected 17 items

t_c28f7427613c42cf86ac12d88a38b3a3.py .................                                      [100%]

======================================= 17 passed in 0.22s ========================================


<ExitCode.OK: 0>

Testklassen müssen mit `Test` beginnen (Groß-/Kleinschreibung beachten!). Fixtures können innerhalb der Klasse definiert werden und stehen dann allen Tests in der Klasse zur Verfügung.

---


---

## 8. Best Practices für gute Tests

### 1. Tests sollen isoliert sein

Jeder Test sollte unabhängig von anderen Tests funktionieren. Keine geteilten Zustände!

In [17]:
from vector import Vector

# SCHLECHT: Tests beeinflussen sich gegenseitig
global_vectors = []

def test_eins():
    global_vectors.append(Vector([1, 2]))
    assert len(global_vectors) == 1

def test_zwei():
    global_vectors.append(Vector([3, 4]))
    assert len(global_vectors) == 1  # Schlägt fehl, wenn test_eins zuerst lief!

# GUT: Jeder Test ist isoliert
@pytest.fixture
def vector_list():
    return []

def test_eins_isolated(vector_list):
    vector_list.append(Vector([1, 2]))
    assert len(vector_list) == 1

def test_zwei_isolated(vector_list):
    vector_list.append(Vector([3, 4]))
    assert len(vector_list) == 1  # Funktioniert immer!

### 2. Ein Test, ein Konzept

Jeder Test sollte genau eine Sache überprüfen:

In [18]:
from vector import Vector

# SCHLECHT: Zu viel in einem Test
def test_vector_operations():
    v = Vector([1, 2, 3])
    assert len(v) == 3
    
    v2 = v * 2
    assert v2 == Vector([2, 4, 6])
    
    v3 = v.normalize()
    assert abs(v3) == pytest.approx(1.0)

# GUT: Separate Tests für verschiedene Konzepte
def test_vector_length():
    v = Vector([1, 2, 3])
    assert len(v) == 3

def test_vector_scalar_multiplication():
    v = Vector([1, 2, 3])
    result = v * 2
    assert result == Vector([2, 4, 6])

def test_vector_normalization():
    v = Vector([1, 2, 3])
    normalized = v.normalize()
    assert abs(normalized) == pytest.approx(1.0)

### 3. Aussagekräftige Testnamen

Der Testname sollte sagen, was getestet wird und was erwartet wird:

In [19]:
# SCHLECHT
def test_1():
    pass

def test_vector():
    pass

# GUT
def test_vector_addition_returns_correct_result():
    pass

def test_vector_division_by_zero_raises_error():
    pass

def test_zero_vector_has_zero_magnitude():
    pass

def test_orthogonal_vectors_have_zero_dot_product():
    pass

### 4. Teste Edge Cases und Grenzwerte

In [20]:
from vector import Vector
import pytest

class TestVectorEdgeCases:
    """Tests für Edge Cases und Grenzwerte"""
    
    def test_empty_vector(self):
        """Leerer Vektor"""
        v = Vector([])
        assert len(v) == 0
        assert abs(v) == 0.0
    
    def test_single_component(self):
        """Vektor mit nur einer Komponente"""
        v = Vector([5])
        assert len(v) == 1
        assert abs(v) == 5.0
    
    def test_zero_vector_boolean(self):
        """Nullvektor ist False"""
        v = Vector([0, 0, 0])
        assert not bool(v)
    
    def test_tiny_components(self):
        """Sehr kleine Komponenten"""
        v = Vector([1e-10, 1e-10])
        assert abs(v) > 0
    
    def test_large_components(self):
        """Sehr große Komponenten"""
        v = Vector([1e100, 1e100])
        assert abs(v) > 0
    
    def test_mixed_signs(self):
        """Gemischte Vorzeichen"""
        v = Vector([-1, 1, -1, 1])
        assert len(v) == 4
    
    @pytest.mark.parametrize("dim", [1, 2, 3, 10, 100])
    def test_various_dimensions(self, dim):
        """Verschiedene Dimensionen"""
        v = Vector([1] * dim)
        assert len(v) == dim

ipytest.run('-v')

======================================= test session starts =======================================
platform win32 -- Python 3.12.4, pytest-8.4.1, pluggy-1.6.0
rootdir: c:\Users\Nils Schillmann\Documents\Sessions\Python_Entwicklung
configfile: pyproject.toml
collected 42 items

t_c28f7427613c42cf86ac12d88a38b3a3.py ...................F......................             [100%]

============================================ FAILURES =============================================
____________________________________________ test_zwei ____________________________________________

    def test_zwei():
        global_vectors.append(Vector([3, 4]))
>       assert len(global_vectors) == 1  # Schlägt fehl, wenn test_eins zuerst lief!
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
E       assert 2 == 1
E        +  where 2 = len([Vector([1.0, 2.0]), Vector([3.0, 4.0])])

C:\Users\Nils Schillmann\AppData\Local\Temp\ipykernel_17896\1104554557.py:12: AssertionError
===================================== short

<ExitCode.TESTS_FAILED: 1>